In [1]:
'''
Population data from FRED API

'''

import requests
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from datetime import datetime
plt.style.use('seaborn-whitegrid')
import numpy as np
import os
from dateutil import parser
from collections import defaultdict

In [2]:

def do_series_search(search_data):
    '''
    Does a search of FRED data series
    INPUT: data to seach (list of strings)
    OUTPUT: json results from FRED
    '''
    fred_search_url = 'https://api.stlouisfed.org/fred/series/search?api_key=4a35874b8a4526dd50b390e46f396651&search_text={0}&file_type=json'  
    if type(search_data) != type(list):
        search_data = [search_data]
    search_string = '+'.join([x.lower() for x in search_data])
    print(search_string)
    fred_response = requests.get(fred_search_url.format(search_string))
    result = fred_response.json()
    return result
    
def get_series(series_id):
    '''
    Fetches FRED data series
    INPUT: data to seach (list of strings)
    OUTPUT: json results from FRED
    '''
    fred_series_fetch_url = 'https://api.stlouisfed.org/fred/series/observations?series_id={0}' + \
'&api_key=4a35874b8a4526dd50b390e46f396651&file_type=json'
    fred_response = requests.get(fred_series_fetch_url.format(series_id))
    result = fred_response.json()
    #print(result)
    return result 


def get_population_data(country):
    '''
    Get Poplatation data from FRED
    INPUT: Country (String)
    OUTPUT: dictionary of data 
    '''
    population_titles_of_interest = [
    'Employment to Population Ratio for',
    'Age Dependency Ratio: Older Dependents to Working-Age Population for',
    'Population Ages 15 to 64 for',
    'Population, Total for',
    'Population'
    ]

    search_string = country.lower()
    
    country_series  = do_series_search(search_string)
    series_ids = {}
    series_notes = {}
    #print(country_series)

    for elem in country_series['seriess']:
        #print(elem)
        for title in population_titles_of_interest:
            #if elem['title'].startswith(title):
            if 'population' in elem['title'].lower():
                series_ids[elem['title']] = elem['id']
                series_notes[elem['title']] = elem['notes']
    print(series_ids)
    ans = {}
    for title, series_id in series_ids.items():
        series = {}
        series_info = get_series(series_id)
        series['title'] = title
        series['units'] = series_info['units']
        series['notes'] = series_notes[title]
        series['dates'] = [x['date'] for x in series_info['observations']]
        series['values'] = [x['value'] for x in series_info['observations']]
        ans[title] = series
    
    return ans


def get_series_info_by_title(title, country):
    '''
    Get series data by specific title
    INPUT: title (str) 
            country (str)
    OUTPUT: series info
    
    '''
    print(title)
    print(country)
    save_country = country
    if country.lower() == 'south korea':
        country = 'republic of korea'
    country_series  = do_series_search(country.lower())
    series_ids = {}
    series_notes = {}
    for elem in country_series['seriess']:
        print(elem['title'] ,elem['frequency'], elem['id'])
        #print(elem)
        if title.lower() == elem['title'].lower():
            if 'frequency' in elem:
                if elem['frequency'].lower() == 'annual':
                    print('found one {0}'.format(title))
                    print(elem)
                    series_ids[elem['title']] = elem['id']
                    series_notes[elem['title']] = elem['notes']
    print('series ids', series_ids)
    ans = {}
    # For some reason cannot generically find us data
    # series_id['population, total for united states'] = 'POPTOTUSA647NWDB'
    # series_notes = {'title':'population, total for united states'}
    for title, series_id in series_ids.items():
        print('search1', title, series_id)
        series = {}
        series_info = get_series(series_id)
        series['title'] = title
        series['units'] = series_info['units']
        series['notes'] = series_notes[title]
        series['dates'] = [x['date'] for x in series_info['observations']]
        series['values'] = [x['value'] for x in series_info['observations']]
        ans[save_country] = series
    return ans
    

        
                
    
    
    

In [3]:
'''
These do not work as the titles vary by country

'''

def get_total_population_data_for_counry_set(country_set):
    '''
    Get total population
    
    '''
    country_set_data = {}
    for country in country_set:
        save_country = country
        country = country.lower()
        title_string = 'population for {0}'.format(country)
        if country.lower() in {'usa', 'united states'}:
            title_string = 'population' 
        print(title_string)
        info = get_series_info_by_title(title_string, save_country)
        country_set_data[save_country] = info
    return country_set_data


'''


'''
def get_working_age_percent_for_country_set(country_set):
    country_set_data = {}
    for country in country_set:
        save_country = country
        country = country.lower()
        title_string = 'population ages 15 to 64 for {0}'.format(country)
        info = get_series_info_by_title(title_string, save_country)
        country_set_data[save_country] = info
    return country_set_data
    
    
    

In [4]:
this_set = get_total_population_data_for_counry_set(['United States'])





population
population
United States
united states
Assets: Total Assets: Total Assets (Less Eliminations from Consolidation): Wednesday Level Weekly, As of Wednesday WALCL
Overnight Reverse Repurchase Agreements: Treasury Securities Sold by the Federal Reserve in the Temporary Open Market Operations Daily RRPONTSYD
Liabilities and Capital: Liabilities: Deposits with F.R. Banks, Other Than Reserve Balances: U.S. Treasury, General Account: Week Average Weekly, Ending Wednesday WTREGEN
Commercial and Industrial Loans, All Commercial Banks Monthly BUSLOANS
Commercial and Industrial Loans, All Commercial Banks Weekly, Ending Wednesday TOTCI
U.S. / Euro Foreign Exchange Rate Daily DEXUSEU
Commercial and Industrial Loans, All Commercial Banks Weekly, Ending Wednesday TOTCINSA
Commercial and Industrial Loans, All Commercial Banks Monthly BUSLOANSNSA
Assets: Securities Held Outright: U.S. Treasury Securities: All: Wednesday Level Weekly, As of Wednesday TREAST
U.S. / Euro Foreign Exchange Rate M

Working Age Population: Aged 15-64: All Persons for the United States Quarterly LFWA64TTUSQ647N
Motor Vehicle Retail Sales: Heavy Weight Trucks Monthly HTRUCKSNSA
ICE BofA AAA US Corporate Index Effective Yield Daily, Close BAMLC0A1CAAAEY
Assets: Securities Held Outright: U.S. Treasury Securities: Maturing in Over 10 Years: Wednesday Level Weekly, As of Wednesday TREAS10Y
Assets: Securities Held Outright: U.S. Treasury Securities: Wednesday Level Weekly, As of Wednesday WSHOTSL
Private Final Consumption Expenditure in United States Annual USAPFCEADSMEI
Overnight Repurchase Agreements: Mortgage-Backed Securities Purchased by the Federal Reserve in the Temporary Open Market Operations Daily RPONMBSD
Germany / U.S. Foreign Exchange Rate (DISCONTINUED) Monthly EXGEUS
Liabilities and Capital: Liabilities: Deposits: U.S. Treasury General Account: Wednesday Level in Federal Reserve District 2: New York Weekly, As of Wednesday D2WLTGAL
Working Age Population: Aged 15-64: All Persons for the Un

Share of Labour Compensation in GDP at Current National Prices for United States Annual LABSHPUSA156NRUG
7-Year Treasury Constant Maturity Rate Monthly GS7
National Totals of State and Local Tax Revenue: Total Taxes for the United States Quarterly QTAXTOTALQTAXCAT1USNO
Median Consumer Price Index Monthly MEDCPIM094SFRBCLE
S&P/Case-Shiller TX-Dallas Home Price Index Monthly DAXRNSA
U.S. / New Zealand Foreign Exchange Rate Monthly EXUSNZ
Median Consumer Price Index Monthly MEDCPIM159SFRBCLE
All Employees, Temporary Help Services Monthly TEMPHELPN
Households and Nonprofit Organizations; Net Worth, Level Annual HNONWRA027N
Unemployment Rate - 20 Yrs. & Over Monthly LNU04000024
U.S. / New Zealand Foreign Exchange Rate Annual AEXUSNZ
Median Consumer Price Index Monthly MEDCPIM157SFRBCLE
Consumer Price Index for All Urban Consumers: New Vehicles in U.S. City Average Semiannual CUUS0000SETA01
Consumer Price Index for All Urban Consumers: Housing in U.S. City Average Monthly CPIHOSSL
Manufactur

In [5]:
this_set

{'United States': {'United States': {'title': 'Population',
   'units': 'lin',
   'notes': 'Population includes resident population plus armed forces overseas. The annual estimate is constructed as an average of monthly series https://fred.stlouisfed.org/series/POPTHM. \nBEA Account Code: B230RC',
   'dates': ['1929-01-01',
    '1930-01-01',
    '1931-01-01',
    '1932-01-01',
    '1933-01-01',
    '1934-01-01',
    '1935-01-01',
    '1936-01-01',
    '1937-01-01',
    '1938-01-01',
    '1939-01-01',
    '1940-01-01',
    '1941-01-01',
    '1942-01-01',
    '1943-01-01',
    '1944-01-01',
    '1945-01-01',
    '1946-01-01',
    '1947-01-01',
    '1948-01-01',
    '1949-01-01',
    '1950-01-01',
    '1951-01-01',
    '1952-01-01',
    '1953-01-01',
    '1954-01-01',
    '1955-01-01',
    '1956-01-01',
    '1957-01-01',
    '1958-01-01',
    '1959-01-01',
    '1960-01-01',
    '1961-01-01',
    '1962-01-01',
    '1963-01-01',
    '1964-01-01',
    '1965-01-01',
    '1966-01-01',
    '196

In [6]:
country_set = ['China', 'Japan', 'Republic of Korea', 'Germany', 'United States']

In [7]:
total_population_set = get_total_population_data_for_counry_set(country_set)

population for china
population for china
China
china
China / U.S. Foreign Exchange Rate Daily DEXCHUS
China / U.S. Foreign Exchange Rate Monthly EXCHUS
China / U.S. Foreign Exchange Rate Annual AEXCHUS
Gross Domestic Product for China Annual MKTGDPCNA646NWDB
Consumer Price Index: All Items for China Monthly CHNCPIALLMINMEI
Consumer Price Index: All Items for China Quarterly CHNCPIALLQINMEI
Consumer Price Index: All Items for China Annual CHNCPIALLAINMEI
M2 for China Monthly MYAGM2CNM189N
Current Price Gross Domestic Product in China Quarterly CHNGDPNQDSMEI
Real GDP at Constant National Prices for China Annual RGDPNACNA666NRUG
Interest Rates, Discount Rate for China Monthly INTDSRCNM193N
Current Price Gross Domestic Product in China Annual CHNGDPNADSMEI
Real Residential Property Prices for China Quarterly QCNR628BIS
Real Residential Property Prices for China Quarterly QCNR368BIS
U.S. Imports of Goods by Customs Basis from China Monthly IMPCH
Constant GDP per capita for China Annual NYG

Ratio of Female to Male Secondary School Enrollment for China Annual SEENRSECOFMZSCHN
Percentage of Foreign Bank Assets Among Total Bank Assets for China Annual DDOI14CNA156NWDB
Bank Lending Deposit Spread for China Annual DDEI02CNA156NWDB
Prices: Consumer Price Index for China, P.R.: Mainland Annual CHNPCPIPCPPPT
Monetary aggregates and their components: Broad money and components: M3: M3 for China Monthly CHNMABMM301GYSAM
Total Current Account Balance for China (DISCONTINUED) Quarterly BPBLTT01CNQ637S
Credit to Private Non-Financial Sector by Banks, Adjusted for Breaks, for China Quarterly, End of Quarter QCNPBM770A
Monetary aggregates and their components: Broad money and components: M3: M3 for China Quarterly CHNMABMM301IXOBSAQ
Monetary aggregates and their components: Broad money and components: M3: M3 for China Monthly CHNMABMM301IXOBSAM
Monetary aggregates and their components: Broad money and components: M3: M3 for China Quarterly CHNMABMM301GYSAQ
Total Electricity Production f

Net Issues of International Debt Securities for All Issuers, All Maturities, Nationality of Issuer in China Quarterly IDSGAMNINICN
Use of Financial Services, Liabilities: Outstanding Deposits at Commercial Banks for China, P.R.: Hong Kong Annual HKGFCLODCXDC
Business tendency surveys (retail trade): Employment: Future tendency: National indicator for China Quarterly CHNBREMFT02STSAQ
Balance of payments BPM6: Financial account: Other investment: Other investment Net incurrence of liabilities for China Quarterly CHNB6FAOI03CXCUQ
Balance of payments BPM6: Financial account: Other investment: Other investment Net incurrence of liabilities for China Quarterly CHNB6FAOI03NCCUQ
Smoothed World Uncertainty Index for Taiwan Province of China Quarterly WUIMATWN
Loans from Non-Resident Banks, Amounts Outstanding, to GDP for China Annual DDOI09CNA156NWDB
Share of Government Consumption at Current Purchasing Power Parities for China Annual CSHGCPCNA156NRUG
Share of Residual Trade and GDP Statistical

Current Account Debit: Total Income for China (DISCONTINUED) Quarterly BPDBIN01CNQ637S
Amount Outstanding of International Money Market instruments for All Issuers, Residence of Issuer in China (DISCONTINUED) Quarterly IMMIRIAOCN
Geographical Outreach: Number of Commercial Banks for China, P.R.: Macao Annual MACFCIODCNUM
Current Account Credit: Total Transfers for China (DISCONTINUED) Quarterly BPCRTF01CNQ637S
Current Account Credit: Total Transfers for China (DISCONTINUED) Annual BPCRTF01CNA637N
Current Account Credit: Total Transfers for China (DISCONTINUED) Annual BPCRTF01CNA637S
Current Account Credit: Total Transfers for China (DISCONTINUED) Quarterly BPCRTF01CNQ637N
Number of Identified Exporters to China from Florida Annual FLCHNA475SCEN
Number of Identified Exporters to China from Texas Annual TXCHNA475SCEN
Amount Outstanding of International Debt Securities for Issuers in Financial Institutions Sector (Banks), All Maturities, Residence of Issuer in China (DISCONTINUED) Quarter

population for japan
population for japan
Japan
japan
Japan / U.S. Foreign Exchange Rate Daily DEXJPUS
Japan / U.S. Foreign Exchange Rate Monthly EXJPUS
Bank of Japan: Total Assets for Japan Monthly, End of Period JPNASSETS
Japan / U.S. Foreign Exchange Rate Annual AEXJPUS
Real Gross Domestic Product for Japan Quarterly JPNRGDPEXP
Residential Property Prices for Japan Quarterly QJPN628BIS
Residential Property Prices for Japan Quarterly QJPN368BIS
Real Gross Domestic Product for Japan Annual NGDPRXDCJPA
Real Gross Domestic Product for Japan Quarterly NGDPRSAXDCJPQ
Real Gross Domestic Product for Japan Quarterly NGDPRNSAXDCJPQ
Long-Term Government Bond Yields: 10-year: Main (Including Benchmark) for Japan Monthly IRLTLT01JPM156N
Gross Domestic Product for Japan Quarterly JPNNGDP
Long-Term Government Bond Yields: 10-year: Main (Including Benchmark) for Japan Annual IRLTLT01JPA156N
Long-Term Government Bond Yields: 10-year: Main (Including Benchmark) for Japan Quarterly IRLTLT01JPQ156N
Con

...

Pension Fund Assets to GDP for Germany Annual DDDI13DEA156NWDB
Production: Manufacturing: Investment goods: Total for Germany Monthly DEUPRMNVG01IXOBSAM
Production: Manufacturing: Intermediate goods: Total for Germany Monthly DEUPRMNIG01IXOBSAM
Production of Total Construction in Germany Monthly DEUPROCONMISMEI
Nominal Final Consumption Expenditure for Germany Quarterly NCSAXDCDEQ
Harmonized Unemployment Rate: All Persons for Germany (DISCONTINUED) Monthly DEUURHARMMDSMEI
Orders: Manufacturing: Total orders: Volume for Germany Monthly DEUODMNTO01IXOBSAM
Ratio of GNP to GDP for Germany Annual GNPGDPDEA156NUPN
Population Growth for Germany Annual SPPOPGROWDEU
Kiel Canal Traffic for Germany Monthly M03021DEM584NNBR
M3, Alternate Definition 2 for Germany Monthly MAM3A2DEM189S
Harmonized Index of Consumer Prices: Medical Services and Paramedical Services for Germany (including former GDR from 1991) Monthly CP0621DEM086NEST
Average Monthly Berlin Rates of Exchange on New York for Germany Mon

Unemployed Population: Aged 15 and Over: All Persons for Germany Annual LFUNTTTTDEA647S
Unemployed Population: Aged 15 and Over: All Persons for Germany Quarterly LFUNTTTTDEQ647N
Unemployed Population: Aged 15 and Over: All Persons for Germany Quarterly LFUNTTTTDEQ647S
Harmonized Unemployment: Total: Females for Germany Annual LRHUTTFEDEA156S
Harmonized Unemployment: Total: Females for Germany Quarterly LRHUTTFEDEQ156N
Consumer Price Index: Transport (COICOP 07): Total: Total for Germany Quarterly DEUCP070000GPQ
Consumer Price Index: Transport (COICOP 07): Total: Total for Germany Quarterly DEUCP070000GYQ
Consumer Price Index: Transport (COICOP 07): Total: Total for Germany Quarterly DEUCP070000IXNBQ
Harmonized Unemployment: Aged 15-24: Males for Germany Annual LFHU24MADEA647N
Harmonized Unemployment: Aged 15-24: Males for Germany Annual LFHU24MADEA647S
Harmonized Unemployment: Aged 15-24: Males for Germany Quarterly LFHU24MADEQ647N
Harmonized Unemployment: Aged 15-24: Males for German

Personal Income Monthly PI
South Africa / U.S. Foreign Exchange Rate Annual AEXSFUS
ICE BofA US High Yield Index Total Return Index Value Daily, Close BAMLHYH0A0HYM2TRIV
Deposits, All Commercial Banks Annual H8B1058NCBCAG
Smoothed U.S. Recession Probabilities Monthly RECPROUSM156N
NBER based Recession Indicators for the United States from the Period following the Peak through the Trough Daily, 7-Day USRECD
Homeownership Rate in the United States Quarterly RSAHORUSQ156S
Federal Surplus or Deficit [-] as Percent of Gross Domestic Product Annual, Fiscal Year FYFSDFYGDP
Producer Price Index by Commodity: Lumber and Wood Products: Softwood Lumber Monthly WPS0811
Consumer Price Index for All Urban Consumers: Rent of Primary Residence in U.S. City Average Monthly CUSR0000SEHA
Thailand / U.S. Foreign Exchange Rate Monthly EXTHUS
Venezuela / U.S. Foreign Exchange Rate Monthly EXVZUS
New One Family Houses Sold: United States Monthly HSN1F
Unemployment Level Monthly UNEMPLOY
Advance Retail Sales:

MZM Money Stock (DISCONTINUED) Monthly MZMSL
Labor Force Participation Rate - 25-54 Yrs. Monthly LNS11300060
Trade Weighted U.S. Dollar Index: Major Currencies, Goods (DISCONTINUED) Annual TWEXMANL
Rental Vacancy Rate in the United States Quarterly RRVRUSQ156N
Housing Inventory Estimate: Total Housing Units in the United States Quarterly ETOTALUSQ176N
Gross National Product Annual GNPA
M1 Money Multiplier (DISCONTINUED) Biweekly, Ending Monday DISMULT
Government total expenditures Annual W068RC1A027NBEA
Trade Weighted U.S. Dollar Index: Advanced Foreign Economies, Goods and Services Daily DTWEXAFEGS
Dates of U.S. recessions as inferred by GDP-based recession indicator Quarterly JHDUSRGDPBR
S&P/Case-Shiller CA-San Diego Home Price Index Monthly SDXRSA
Real-time Sahm Rule Recession Indicator Monthly SAHMREALTIME
Total Consumer Credit Owned and Securitized Monthly TOTALNS
US Regular All Formulations Gas Price Monthly, End of Period GASREGM
Sweden / U.S. Foreign Exchange Rate Daily DEXSDUS

Personal Consumption Expenditures: Nondurable Goods Quarterly PCND
Delinquency Rate on Commercial and Industrial Loans, All Commercial Banks Quarterly, End of Period DRBLACBN
4-Week Treasury Bill: Secondary Market Rate Daily DTB4WK
Consumer Loans, All Commercial Banks Monthly H8B1029NCBCMG
S&P/Case-Shiller GA-Atlanta Home Price Index Monthly ATXRSA
Liabilities and Capital: Liabilities: Deposits (Less Eliminations from Consolidation): Wednesday Level Weekly, As of Wednesday WLDLCL
Term Premium on a 10 Year Zero Coupon Bond Daily THREEFYTP10
Personal Consumption Expenditures (PCE) Excluding Food and Energy (chain-type price index) Quarterly DPCCRV1Q225SBEA
State Minimum Wage Rate for Pennsylvania Annual STTMINWGPA
All-Transactions House Price Index for Los Angeles County, CA Annual ATNHPIUS06037A
Federal Minimum Hourly Wage for Nonfarm Workers for the United States Monthly FEDMINNFRWG
State Minimum Wage Rate for Texas Annual STTMINWGTX
ICE BofA 7-10 Year US Corporate Index Effective Yiel

In [8]:

total_population_set

{'China': {'China': {'title': 'Population for China',
   'units': 'lin',
   'notes': 'Source ID: pop\n\nWhen using these data in your research, please make the following reference: Feenstra, Robert C., Robert Inklaar and Marcel P. Timmer (2015), "The Next Generation of the Penn World Table" American Economic Review, 105(10), 3150-3182, available for download at www.ggdc.net/pwt\n\nFor more information, see http://www.rug.nl/research/ggdc/data/pwt/.',
   'dates': ['1952-01-01',
    '1953-01-01',
    '1954-01-01',
    '1955-01-01',
    '1956-01-01',
    '1957-01-01',
    '1958-01-01',
    '1959-01-01',
    '1960-01-01',
    '1961-01-01',
    '1962-01-01',
    '1963-01-01',
    '1964-01-01',
    '1965-01-01',
    '1966-01-01',
    '1967-01-01',
    '1968-01-01',
    '1969-01-01',
    '1970-01-01',
    '1971-01-01',
    '1972-01-01',
    '1973-01-01',
    '1974-01-01',
    '1975-01-01',
    '1976-01-01',
    '1977-01-01',
    '1978-01-01',
    '1979-01-01',
    '1980-01-01',
    '1981-01-0

In [9]:
working_population_set = get_working_age_percent_for_country_set(country_set)

population ages 15 to 64 for china
China
china
China / U.S. Foreign Exchange Rate Daily DEXCHUS
China / U.S. Foreign Exchange Rate Monthly EXCHUS
China / U.S. Foreign Exchange Rate Annual AEXCHUS
Gross Domestic Product for China Annual MKTGDPCNA646NWDB
Consumer Price Index: All Items for China Monthly CHNCPIALLMINMEI
Consumer Price Index: All Items for China Quarterly CHNCPIALLQINMEI
Consumer Price Index: All Items for China Annual CHNCPIALLAINMEI
M2 for China Monthly MYAGM2CNM189N
Current Price Gross Domestic Product in China Quarterly CHNGDPNQDSMEI
Real GDP at Constant National Prices for China Annual RGDPNACNA666NRUG
Interest Rates, Discount Rate for China Monthly INTDSRCNM193N
Current Price Gross Domestic Product in China Annual CHNGDPNADSMEI
Real Residential Property Prices for China Quarterly QCNR628BIS
Real Residential Property Prices for China Quarterly QCNR368BIS
U.S. Imports of Goods by Customs Basis from China Monthly IMPCH
Constant GDP per capita for China Annual NYGDPPCAPK

Goods, Value of Exports for China Monthly VALEXPCNM052N
Import Price Index by Origin (NAICS): Electric Lighting Equipment Manufacturing for China Monthly COCHNZ3351
Credit to Private Non-Financial Sector by Banks, Adjusted for Breaks, for China Quarterly, End of Quarter QCNPBMUSDA
Net migration for China 5 Year SMPOPNETMCHN
Total Electricity Production for China Quarterly PRENEL01CNQ656N
Total Credit to Non-Financial Corporations, Unadjusted for Breaks, for China Quarterly, End of Quarter CRDQCNANUBIS
Bank's Return on Equity for China Annual DDEI06CNA156NWDB
Bank Capital to Total Assets for China Annual DDSI03CNA156NWDB
Balance of payments BPM6: Financial account: Reserve assets: Reserve assets Net for China Quarterly CHNB6FARA01CXCUQ
Bank Concentration for China Annual DDOI01CNA156NWDB
Mobile Cellular Subscriptions in China Annual ITCELSETSP2CHN
Liquid Liabilities to GDP for China Annual DDDI05CNA156NWDB
Import Price Index by Origin (NAICS): Manufacturing, Part 1 for China Monthly COC

Exports: Value Goods to European Union (15 countries) from China (DISCONTINUED) Quarterly XTEXVA04CNQ667N
Exports: Value Goods to European Union (15 countries) from China (DISCONTINUED) Annual XTEXVA04CNA667N
Imports: Value Goods from European Union (15 countries) to China (DISCONTINUED) Monthly XTIMVA04CNM667N
Imports: Value Goods from European Union (15 countries) to China (DISCONTINUED) Quarterly XTIMVA04CNQ667N
Imports: Value Goods from European Union (15 countries) to China (DISCONTINUED) Annual XTIMVA04CNA667N
Openness at Current Prices for China Annual OPENC2CNA156NUPN
Openness at Current Prices for China Annual OPENCPCNA156NUPN
Government Consumption Share of Purchasing Power Parity Converted GDP Per Capita at current prices for China Annual CGPPPGCNA156NUPN
Government Consumption Share of Purchasing Power Parity Converted GDP Per Capita at current prices for China Annual CGPPP2CNA156NUPN
Government Consumption Share of Purchasing Power Parity Converted GDP Per Capita at consta

Share of Merchandise Exports at Current Purchasing Power Parities for China Annual CSHXCPCNA156NRUG
Use of Financial Services, Assets: Outstanding Loans to Small and Medium Enterprises at Commercial Banks for China, P.R.: Mainland Annual CHNFCSODCSXDC
Financial Account: Net Financial Account Excluding Change in Official Reserves for China (DISCONTINUED) Quarterly BPFATT02CNQ637N
Financial Account: Net Financial Account Excluding Change in Official Reserves for China (DISCONTINUED) Annual BPFATT02CNA637N
Use of Financial Services Borrowers: Households Borrowing from Commercial Banks for China, P.R.: Mainland Annual CHNFCRODCHPENUM
Geographical Outreach: Key Indicators Commercial Bank Branches Per 100,000 Adults for China, P.R.: Hong Kong Annual HKGFCBODCANUM
External Trade: Volume of Imports of Goods and Services for China, P.R.: Hong Kong Annual HKGTMRPCPPPT
Loans from Non-Resident Banks, Net, to GDP for China Annual DDOI08CNA156NWDB
Price level of Output-side Real GDP at Current Purch

Geographical Outreach: Key Indicators ATMs Per 100,000 Adults for China, P.R.: Macao Annual MACFCAANUM
Current Account Credit: Total Services for China (DISCONTINUED) Quarterly BPCRSE01CNQ637S
Gross Issuance of International Money Market instruments for All Issuers, Nationality of Issuer in China (DISCONTINUED) Quarterly IMMINIAIGICN
Use of Financial Services: Number of Depositors at Other Depository Corporations for China, P.R.: Mainland Annual CHNFCDODPENUM
Net Issues of International Debt Securities for Issuers in General Government Sector, All Maturities, Nationality of Issuer in China (DISCONTINUED) Quarterly IDSGGGAMNINICN
Amount Outstanding of International Debt Securities for Issuers in Non-Financial Corporations (Corporate Issuers), All Maturities, Nationality of Issuer in China (DISCONTINUED) Quarterly IDSGNFAMNIAOCN
Geographical Outreach: Number of Automated Teller Machines (ATMs), Country Wide for China, P.R.: Hong Kong Annual HKGFCACNUM
Geographical Outreach: Number of Ins

...

Harmonized Index of Consumer Prices: Overall Index Excluding Alcohol and Tobacco for Germany (including former GDR from 1991) Monthly 00XALCDEM086NEST
Male Applicants Per Hundred Positions for Germany Monthly M08028DEM468NNBR
Harmonized Index of Consumer Prices: Electricity for Germany (including former GDR from 1991) Monthly CP0451DEM086NEST
Wholesale Price of Coal for Essen, Germany Monthly M04I1ADE00ESSM372NNBR
Liquid Liabilities to GDP for Germany Annual DDDI05DEA156NWDB
Harmonized Index of Consumer Prices: Motor Cars for Germany (including former GDR from 1991) Monthly CP0711DEM086NEST
Import Price Index by Origin (NAICS): Machinery Manufacturing for Germany Monthly COGERZ333
Bank Z-Score for Germany Annual DDSI01DEA645NWDB
Average Monthly Berlin Rates of Exchange on Paris for Germany Monthly M14071DEM369NNBR
Price Level of Household Consumption for Germany Annual PLCCPPDEA670NRUG
Early Estimate of Quarterly ULC Indicators: Total Labor Productivity for Germany Quarterly ULQELP01DE

Harmonized Unemployment: Aged 15-24: All Persons for Germany Quarterly LRHU24TTDEQ156S
Harmonized Unemployment: Aged 15-24: All Persons for Germany Quarterly LRHU24TTDEQ156N
Consumer Price Index: Food for Germany Annual DEUCPIFODAINMEI
Consumer Price Index: Food for Germany Quarterly DEUCPIFODQINMEI
Unemployed Population: Aged 15 and Over: All Persons for Germany Annual LFUNTTTTDEA647N
Production: Manufacturing: Investment goods: Total for Germany Monthly DEUPRMNVG01IXOBM
Production: Manufacturing: Intermediate goods: Total for Germany Monthly DEUPRMNIG01IXOBM
Production of Total Construction in Germany Annual DEUPROCONAISMEI
Production of Total Construction in Germany Quarterly DEUPROCONQISMEI
Harmonized Unemployment Rate: All Persons for Germany (DISCONTINUED) Annual DEUURHARMADSMEI
Harmonized Unemployment Rate: All Persons for Germany (DISCONTINUED) Quarterly DEUURHARMQDSMEI
Harmonized Unemployment: Total: Females for Germany Monthly LRHUTTFEDEM156S
Harmonized Unemployment: Total: F

Assets: Total Assets: Total Assets (Less Eliminations from Consolidation): Wednesday Level Weekly, As of Wednesday WALCL
Overnight Reverse Repurchase Agreements: Treasury Securities Sold by the Federal Reserve in the Temporary Open Market Operations Daily RRPONTSYD
Liabilities and Capital: Liabilities: Deposits with F.R. Banks, Other Than Reserve Balances: U.S. Treasury, General Account: Week Average Weekly, Ending Wednesday WTREGEN
Commercial and Industrial Loans, All Commercial Banks Monthly BUSLOANS
Commercial and Industrial Loans, All Commercial Banks Weekly, Ending Wednesday TOTCI
U.S. / Euro Foreign Exchange Rate Daily DEXUSEU
Commercial and Industrial Loans, All Commercial Banks Weekly, Ending Wednesday TOTCINSA
Commercial and Industrial Loans, All Commercial Banks Monthly BUSLOANSNSA
Assets: Securities Held Outright: U.S. Treasury Securities: All: Wednesday Level Weekly, As of Wednesday TREAST
U.S. / Euro Foreign Exchange Rate Monthly EXUSEU
U.S. / Euro Foreign Exchange Rate An

S&P/Case-Shiller CA-San Francisco Home Price Index Monthly SFXRSA
E-Commerce Retail Sales as a Percent of Total Sales Quarterly ECOMPCTNSA
1-Month Treasury Constant Maturity Rate Monthly GS1M
Delinquency Rate on Credit Card Loans, All Commercial Banks Quarterly, End of Period DRCCLACBN
Moody's Seasoned Aaa Corporate Bond Yield Relative to Yield on 10-Year Treasury Constant Maturity Monthly AAA10YM
Federal Debt Held by Federal Reserve Banks Quarterly, End of Period FDHBFRBN
Real Disposable Personal Income: Per Capita Annual A229RX0A048NBEA
3-Year Treasury Constant Maturity Rate Monthly GS3
New One Family Houses Sold: United States Annual HSN1FA
Trade Balance: Goods and Services, Balance of Payments Basis Monthly BOPGSTB
Employment Cost Index: Wages and Salaries: Private Industry Workers Quarterly ECIWAG
Private Final Consumption Expenditure in United States Quarterly USAPFCEQDSNAQ
Real Disposable Personal Income: Per Capita Quarterly A229RX0Q048SBEA
3-Year Treasury Constant Maturity Rat

Median Sales Price for New Houses Sold in the United States Monthly MSPNHSUS
Gross Federal Debt as Percent of Gross Domestic Product Annual GFDGDPA188S
Population Level Monthly CNP16OV
Book Value of Federal Reserve Bank Held Gold Bullion: NY Vault Monthly FRVGBSV
Producer Price Index by Commodity: Final Demand: Finished Goods Less Foods and Energy Monthly WPSFD4131
Unemployment Rate in California Annual LAUST060000000000003A
Unemployment Rate in Florida Monthly FLURN
Federal Government: Current Expenditures Quarterly FGEXPND
Federal Debt Held by the Public Quarterly, End of Period FYGFDPUN
E-Commerce Retail Sales Quarterly ECOMSA
ICE BofA US Corporate Index Effective Yield Daily, Close BAMLC0A0CMEY
Monetary Base; Currency in Circulation Monthly MBCURRCIR
Consumer Price Index for All Urban Consumers: Housing in U.S. City Average Monthly CPIHOSNS
Reverse Repurchase Agreements: Total Securities Sold by the Federal Reserve in the Temporary Open Market Operations Daily RRPTTLD
Manufacturing

In [10]:
working_population_set

{'China': {'China': {'title': 'Population Ages 15 to 64 for China',
   'units': 'lin',
   'notes': 'Source Indicator: SP.POP.1564.TO.ZS',
   'dates': ['1950-01-01',
    '1951-01-01',
    '1952-01-01',
    '1953-01-01',
    '1954-01-01',
    '1955-01-01',
    '1956-01-01',
    '1957-01-01',
    '1958-01-01',
    '1959-01-01',
    '1960-01-01',
    '1961-01-01',
    '1962-01-01',
    '1963-01-01',
    '1964-01-01',
    '1965-01-01',
    '1966-01-01',
    '1967-01-01',
    '1968-01-01',
    '1969-01-01',
    '1970-01-01',
    '1971-01-01',
    '1972-01-01',
    '1973-01-01',
    '1974-01-01',
    '1975-01-01',
    '1976-01-01',
    '1977-01-01',
    '1978-01-01',
    '1979-01-01',
    '1980-01-01',
    '1981-01-01',
    '1982-01-01',
    '1983-01-01',
    '1984-01-01',
    '1985-01-01',
    '1986-01-01',
    '1987-01-01',
    '1988-01-01',
    '1989-01-01',
    '1990-01-01',
    '1991-01-01',
    '1992-01-01',
    '1993-01-01',
    '1994-01-01',
    '1995-01-01',
    '1996-01-01',
    '19

In [11]:

working_age_population_set = {
'Germany':'Working Age Population: Aged 15-64: All Persons for Germany',
'South Korea':'Working Age Population: Aged 15-64: All Persons for the Republic of Korea',
'Japan':'Working Age Population: Aged 15-64: All Persons for Japan',
'USA':'Working Age Population: Aged 15-64: All Persons for the United States',
'China':'Population Ages 15 to 64 for China', # This is a ratio -- need to multiple by population
}

working_age_population_ratio_set = {
'Germany':'Employment to Population Ratio for Germany',
'South Korea':'Employment to Population Ratio for the Republic of Korea',
'Japan':'Employment to Population Ratio for Japan',
'USA':'Employment to Population Ratio for the United States', # USA
'China':'Population Ages 15 to 64 for China',
}


In [12]:
working_age_population = {}
for country in working_age_population_set:
    working_age_population[country] =  get_series_info_by_title( working_age_population_set[country], country )

Working Age Population: Aged 15-64: All Persons for Germany
Germany
germany
Long-Term Government Bond Yields: 10-year: Main (Including Benchmark) for Germany Monthly IRLTLT01DEM156N
Long-Term Government Bond Yields: 10-year: Main (Including Benchmark) for Germany Quarterly IRLTLT01DEQ156N
Long-Term Government Bond Yields: 10-year: Main (Including Benchmark) for Germany Annual IRLTLT01DEA156N
Real Gross Domestic Product for Germany Quarterly CLVMNACSCAB1GQDE
Real Gross Domestic Product for Germany Quarterly NGDPRSAXDCDEQ
Real Gross Domestic Product for Germany Quarterly CLVMNACNSAB1GQDE
Real Gross Domestic Product for Germany Annual NGDPRXDCDEA
Real Gross Domestic Product for Germany Quarterly NGDPRNSAXDCDEQ
Consumer Price Index of All Items in Germany Monthly DEUCPIALLMINMEI
Consumer Price Index of All Items in Germany Quarterly DEUCPIALLQINMEI
Gross Domestic Product for Germany Quarterly CPMNACSCAB1GQDE
Registered Unemployment Rate for Germany Monthly LMUNRRTTDEM156S
Registered Unempl

Production: Manufacturing: Consumer goods: Durable goods for Germany Monthly DEUPRMNCG02IXOBSAM
Amount Outstanding of International Bonds and Notes for All Issuers, Nationality of Issuer in Germany (DISCONTINUED) Quarterly IBANAINIAODE
Private Final Consumption Expenditure Implicit Price Deflator in Germany Quarterly DEUPCEDEFQISNAQ
Index of Sensitive Prices for Germany Monthly M04056DEM324NNBR
Harmonized Index of Consumer Prices: Overall Index Excluding Alcohol and Tobacco for Germany (including former GDR from 1991) Monthly 00XALCDEM086NEST
Male Applicants Per Hundred Positions for Germany Monthly M08028DEM468NNBR
Harmonized Index of Consumer Prices: Electricity for Germany (including former GDR from 1991) Monthly CP0451DEM086NEST
Wholesale Price of Coal for Essen, Germany Monthly M04I1ADE00ESSM372NNBR
Liquid Liabilities to GDP for Germany Annual DDDI05DEA156NWDB
Harmonized Index of Consumer Prices: Motor Cars for Germany (including former GDR from 1991) Monthly CP0711DEM086NEST
Impo

Harmonized Unemployment: Aged 15-24: All Persons for Germany Monthly LFHU24TTDEM647S
Harmonized Unemployment: Aged 15-24: All Persons for Germany Monthly LRHU24TTDEM156S
Harmonized Unemployment: Aged 15-24: All Persons for Germany Monthly LFHU24TTDEM647N
Harmonized Unemployment: Aged 15-24: All Persons for Germany Monthly LRHU24TTDEM156N
Harmonized Unemployment: Aged 15-24: All Persons for Germany Quarterly LRHU24TTDEQ156S
Harmonized Unemployment: Aged 15-24: All Persons for Germany Quarterly LRHU24TTDEQ156N
Consumer Price Index: Food for Germany Annual DEUCPIFODAINMEI
Consumer Price Index: Food for Germany Quarterly DEUCPIFODQINMEI
Unemployed Population: Aged 15 and Over: All Persons for Germany Annual LFUNTTTTDEA647N
Production: Manufacturing: Investment goods: Total for Germany Monthly DEUPRMNVG01IXOBM
Production: Manufacturing: Intermediate goods: Total for Germany Monthly DEUPRMNIG01IXOBM
Production of Total Construction in Germany Annual DEUPROCONAISMEI
Production of Total Constr

Working Age Population: Aged 15-64: All Persons for the Republic of Korea
South Korea
republic of korea
South Korea / U.S. Foreign Exchange Rate Daily DEXKOUS
South Korea / U.S. Foreign Exchange Rate Monthly EXKOUS
South Korea / U.S. Foreign Exchange Rate Annual AEXKOUS
Gross Domestic Product by Expenditure in Constant Prices: Total Gross Domestic Product for the Republic of Korea Quarterly NAEXKP01KRQ189S
Consumer Price Index: All Items for Korea Monthly KORCPIALLMINMEI
Gross Domestic Product by Expenditure in Constant Prices: Total Gross Domestic Product for the Republic of Korea Quarterly NAEXKP01KRQ657S
Gross Domestic Product by Expenditure in Constant Prices: Total Gross Domestic Product for the Republic of Korea Annual NAEXKP01KRA657S
Gross Domestic Product by Expenditure in Constant Prices: Total Gross Domestic Product for the Republic of Korea Quarterly NAEXKP01KRQ661S
Gross Domestic Product by Expenditure in Constant Prices: Total Gross Domestic Product for the Republic of Kor

Active Population: Aged 15 and Over: Females for the Republic of Korea Monthly LFACTTFEKRM647S
Inactive Population: Aged 55-64: All Persons for the Republic of Korea Monthly LFIN55TTKRM647S
Inactive Population: Aged 55-64: All Persons for the Republic of Korea Monthly LFIN55TTKRM647N
Producer Prices Index: Economic Activities: Domestic Industrial Activities for the Republic of Korea Annual PIEATI02KRA661N
Producer Prices Index: Economic Activities: Domestic Industrial Activities for the Republic of Korea Quarterly PIEATI02KRQ661N
Inactive Population: Aged 15-64: Males for the Republic of Korea Monthly LFIN64MAKRM647S
Inactive Population: Aged 15-64: Males for the Republic of Korea Monthly LFIN64MAKRM647N
Employment by Economic Activity: Manufacturing: All Persons for the Republic of Korea Monthly LFEAMNTTKRM647S
Employment by Economic Activity: Manufacturing: All Persons for the Republic of Korea Monthly LFEAMNTTKRM647N
Employment by Economic Activity: Manufacturing: All Persons for th

...

Consumer Loans: Other Consumer Loans: Automobile Loans, All Commercial Banks Weekly CARACBW027SBOG
Unemployment Level Monthly LNU03000000
Population Monthly POPTHM
ICE BofA AA US Corporate Index Effective Yield Daily, Close BAMLC0A2CAAEY
Liabilities and Capital: Liabilities: Reverse Repurchase Agreements: Foreign Official and International Accounts: Wednesday Level Weekly, As of Wednesday WLRRAFOIAL
Personal Saving Monthly PMSAVE
Weekly Economic Index (Lewis-Mertens-Stock) Weekly, Ending Saturday WEI
Amount Outstanding of Total Debt Securities in Non-Financial Corporations Sector, All Maturities, Residence of Issuer in United States Quarterly TDSAMRIAONCUS
Existing Home Sales: Housing Inventory Monthly HOSINVUSM495N
Stock Market Capitalization to GDP for United States Annual DDDM01USA156NWDB
Housing Inventory: Active Listing Count in the United States Monthly ACTLISCOUUS
S&P/Case-Shiller CA-San Francisco Home Price Index Monthly SFXRSA
E-Commerce Retail Sales as a Percent of Total Sale

Housing Inventory: Median Listing Price per Square Feet in the United States Monthly MEDLISPRIPERSQUFEEUS
U.S. / New Zealand Foreign Exchange Rate Daily DEXUSNZ
Consumer Price Index: All Items for the United States Monthly USACPIALLMINMEI
Large Time Deposits, All Commercial Banks Monthly LTDACBM027SBOG
Large Time Deposits, All Commercial Banks Weekly, Ending Wednesday LTDACBW027NBOG
Delinquency Rate on Consumer Loans, All Commercial Banks Quarterly, End of Period DRCLACBN
Net exports of goods and services Quarterly NA000374Q
Net exports of goods and services Annual A019RC1A027NBEA
New Privately-Owned Housing Units Started: Single-Family Units Monthly HOUST1FNSA
M2 for United States Monthly MYAGM2USM052N
Henry Hub Natural Gas Spot Price Annual AHHNGSP
Consumer Price Index for All Urban Consumers: Owners' Equivalent Rent of Residences in U.S. City Average Monthly CUSR0000SEHC
Median Sales Price for New Houses Sold in the United States Monthly MSPNHSUS
Gross Federal Debt as Percent of Gro

Real Estate Loans, All Commercial Banks Annual H8B1026NCBCAG
Money Market Funds; Total Financial Assets, Transactions Annual MMMFTAA027N
Personal income per capita Annual A792RC0A052NBEA
Money Market Funds; Total Financial Assets, Transactions Annual BOGZ1FU634090005A
Personal Consumption Expenditures: Services Quarterly NA000350Q
Real Imports of Goods and Services Quarterly A021RL1Q158SBEA
Balance on current account Annual IEABCA
Real Imports of Goods and Services Quarterly A021RO1Q156NBEA
Import Price Index (End Use): All Commodities Monthly IR
Unemployment Rate for United States Monthly M0892CUSM156NNBR
Personal Consumption Expenditures: Services Quarterly NC000350Q
Real Imports of Goods and Services Annual A021RL1A158NBEA
Total wages and salaries, BLS Annual BA06RC1A027NBEA
Number Unemployed for 27 Weeks & Over Monthly LNU03008636
Real Imports of Goods and Services Quarterly NB000342Q
Real Imports of Goods and Services Quarterly OB000342Q
Interest Rates and Price Indexes; Commercia

Reserve Assets for China (DISCONTINUED) Annual XFORSD01CNA194N
Reserve Assets for China (DISCONTINUED) Quarterly XFORSD01CNQ194N
Current Account (Excludes Exceptional Financing), Net for China Annual CUAEEFCNA052N
Broad Money, Index for China (DISCONTINUED) Monthly MABMBM01CNM657S
Broad Money, Index for China (DISCONTINUED) Monthly MABMBM01CNM659S
Broad Money, Index for China (DISCONTINUED) Monthly MABMBM01CNM661S
Broad Money, Index for China (DISCONTINUED) Annual MABMBM01CNA659S
Broad Money, Index for China (DISCONTINUED) Quarterly MABMBM01CNQ659S
Broad Money, Index for China (DISCONTINUED) Annual MABMBM01CNA657S
Broad Money, Index for China (DISCONTINUED) Quarterly MABMBM01CNQ661S
Broad Money, Index for China (DISCONTINUED) Annual MABMBM01CNA661S
Broad Money, Index for China (DISCONTINUED) Quarterly MABMBM01CNQ657S
Current Account Credit: Trade: Total Exports of Goods for China (DISCONTINUED) Annual BPCRTD01CNA637N
Current Account Credit: Trade: Total Exports of Goods for China (DISC

Net Issues of International Debt Securities for Issuers in Non-Financial Corporations (Corporate Issuers), All Maturities, Nationality of Issuer in China (DISCONTINUED) Quarterly IDSGNFAMNINICN
Value of Exports to China from North Dakota Annual NDCHNA052SCEN
Value of Exports to China from Alaska Annual AKCHNA052SCEN
Geographical Outreach: Number of Automated Teller Machines (ATMs), Country Wide for China, P.R.: Macao Annual MACFCACNUM
Use of Financial Services, Liabilities: Outstanding Deposits at Insurance Corporations for China, P.R.: Mainland Annual CHNFCLOFIXDC
Use of Financial Services: Number of Depositors at Credit Unions and Financial Cooperatives for China, P.R.: Mainland Annual CHNFCDODUPENUM
Number of Identified Exporters to China from New Mexico Annual NMCHNA475SCEN
Geographical Outreach: Number of Institutions, Other Financial Corporations, Insurance Corporations for China, P.R.: Mainland Annual CHNFCIOFINUM
Value of Exports to China from Hawaii Annual HICHNA052SCEN
Value 

In [13]:
working_age_population_ratio = {}
for country in working_age_population_ratio_set:
    working_age_population_ratio[country] = get_series_info_by_title(working_age_population_ratio_set[country], country )

Employment to Population Ratio for Germany
Germany
germany
Long-Term Government Bond Yields: 10-year: Main (Including Benchmark) for Germany Monthly IRLTLT01DEM156N
Long-Term Government Bond Yields: 10-year: Main (Including Benchmark) for Germany Quarterly IRLTLT01DEQ156N
Long-Term Government Bond Yields: 10-year: Main (Including Benchmark) for Germany Annual IRLTLT01DEA156N
Real Gross Domestic Product for Germany Quarterly CLVMNACSCAB1GQDE
Real Gross Domestic Product for Germany Quarterly NGDPRSAXDCDEQ
Real Gross Domestic Product for Germany Quarterly CLVMNACNSAB1GQDE
Real Gross Domestic Product for Germany Annual NGDPRXDCDEA
Real Gross Domestic Product for Germany Quarterly NGDPRNSAXDCDEQ
Consumer Price Index of All Items in Germany Monthly DEUCPIALLMINMEI
Consumer Price Index of All Items in Germany Quarterly DEUCPIALLQINMEI
Gross Domestic Product for Germany Quarterly CPMNACSCAB1GQDE
Registered Unemployment Rate for Germany Monthly LMUNRRTTDEM156S
Registered Unemployment Rate for G

Labor Force Participation Rate for Men in Germany (DISCONTINUED) Annual DEULFPMNA
Fertility Rate, Total for Germany Annual SPDYNTFRTINDEU
Number of Residential Buildings Constructed and Inspected for Berlin, Germany Monthly M0080ADE00BERM470NNBR
Producer Prices Index: Domestic Investments Goods for Germany Monthly PITGVG02DEM661N
Wholesale Price of Potatoes, Average Per Month for Berlin, Germany Monthly M04116DE00BERM372NNBR
Producer Prices Index: Economic Activities: Total Industrial Activities for Germany Monthly PIEATI01DEM661N
Infant Mortality Rate for Germany Annual SPDYNIMRTINDEU
Consumer Opinion Surveys: Consumer Prices: Future Tendency of Inflation: European Commission and National Indicators for Germany Monthly CSINFT02DEM460S
Real Final Consumption Expenditure for Germany Quarterly NCRSAXDCDEQ
Current Account Balance: Total Trade of Goods for Germany (DISCONTINUED) Quarterly BPBLTD01DEQ637S
Leading Indicators OECD: Component series: Interest rate spread: Normalised for German

Consumer Price Index: Housing, water, electricity, gas and other fuels (COICOP 04): Total: Total for Germany Monthly DEUCP040000CTGYM
Consumer Price Index: Housing, water, electricity, gas and other fuels (COICOP 04): Total: Total for Germany Monthly DEUCP040000GYM
Consumer Price Index: Housing, water, electricity, gas and other fuels (COICOP 04): Total: Total for Germany Monthly DEUCP040000GPM
Consumer Price Index: Housing, water, electricity, gas and other fuels (COICOP 04): Total: Total for Germany Monthly DEUCP040000IXOBM
Consumer Price Index: Housing, water, electricity, gas and other fuels (COICOP 04): Total: Total for Germany Monthly DEUCP040000IXNBM
Consumer Price Index: Housing, water, electricity, gas and other fuels (COICOP 04): Total: Total for Germany Quarterly DEUCP040000GPQ
Consumer Price Index: Housing, water, electricity, gas and other fuels (COICOP 04): Total: Total for Germany Quarterly DEUCP040000GYQ
Consumer Price Index: Housing, water, electricity, gas and other f

Employment to Population Ratio for the Republic of Korea
South Korea
republic of korea
South Korea / U.S. Foreign Exchange Rate Daily DEXKOUS
South Korea / U.S. Foreign Exchange Rate Monthly EXKOUS
South Korea / U.S. Foreign Exchange Rate Annual AEXKOUS
Gross Domestic Product by Expenditure in Constant Prices: Total Gross Domestic Product for the Republic of Korea Quarterly NAEXKP01KRQ189S
Consumer Price Index: All Items for Korea Monthly KORCPIALLMINMEI
Gross Domestic Product by Expenditure in Constant Prices: Total Gross Domestic Product for the Republic of Korea Quarterly NAEXKP01KRQ657S
Gross Domestic Product by Expenditure in Constant Prices: Total Gross Domestic Product for the Republic of Korea Annual NAEXKP01KRA657S
Gross Domestic Product by Expenditure in Constant Prices: Total Gross Domestic Product for the Republic of Korea Quarterly NAEXKP01KRQ661S
Gross Domestic Product by Expenditure in Constant Prices: Total Gross Domestic Product for the Republic of Korea Annual NAEXKP0

Employment Rate: Aged 15 and Over: All Persons for the Republic of Korea Quarterly LREMTTTTKRQ156N
Employment Rate: Aged 15 and Over: All Persons for the Republic of Korea Annual LREMTTTTKRA156N
Consumer Price Index: Alcoholic beverages, tobacco and narcotics (COICOP 02): Total: Total for the Republic of Korea Monthly KORCP020000GPM
Consumer Price Index: Alcoholic beverages, tobacco and narcotics (COICOP 02): Total: Total for the Republic of Korea Monthly KORCP020000CTGYM
Consumer Price Index: Alcoholic beverages, tobacco and narcotics (COICOP 02): Total: Total for the Republic of Korea Monthly KORCP020000GYM
Consumer Price Index: Alcoholic beverages, tobacco and narcotics (COICOP 02): Total: Total for the Republic of Korea Monthly KORCP020000IXOBM
Consumer Price Index: Alcoholic beverages, tobacco and narcotics (COICOP 02): Total: Total for the Republic of Korea Monthly KORCP020000IXNBM
Consumer Price Index: Alcoholic beverages, tobacco and narcotics (COICOP 02): Total: Total for the 

...

New One Family Houses Sold: United States Monthly HSN1FNSA
Total Unemployed, Plus All Persons Marginally Attached to the Labor Force, Plus Total Employed Part Time for Economic Reasons, as a Percent of the Civilian Labor Force Plus All Persons Marginally Attached to the Labor Force (U-6) Monthly U6RATENSA
Unemployment Rate - 20 Yrs. & Over, Black or African American Men Monthly LNU04000031
Average Hourly Earnings of Production and Nonsupervisory Employees, Total Private Monthly AHETPI
All Sectors; Debt Securities and Loans; Liability, Level Quarterly, End of Period TCMDO
4-Week Moving Average of Initial Claims Weekly, Ending Saturday IC4WSA
Consumer Price Index for All Urban Consumers: Rent of Primary Residence in U.S. City Average Semiannual CUUS0000SEHA
Excess Reserves of Depository Institutions (DISCONTINUED) Monthly EXCRESNS
Share of Total Net Worth Held by the Top 1% (99th to 100th Wealth Percentiles) Quarterly WFRBST01134
Capacity Utilization: Total Index Quarterly CAPUTLB50001SQ

Total Savings Deposits at all Depository Institutions (DISCONTINUED) Weekly, Ending Monday WSAVNS
FOMC Summary of Economic Projections for the Fed Funds Rate, Median Annual FEDTARMD
National Totals of State and Local Tax Revenue: Total Taxes for the United States Quarterly QTAXTOTALQTAXCAT1USYES
ICE BofA AAA US Corporate Index Option-Adjusted Spread Daily, Close BAMLC0A1CAAA
ICE BofA AA US Corporate Index Option-Adjusted Spread Daily, Close BAMLC0A2CAA
Assets: Securities Held Outright: Securities Held Outright: Week Average Weekly, Ending Wednesday WSECOUT
Median Sales Price of Existing Homes Monthly HOSMEDUSM052N
S&P/Case-Shiller IL-Chicago Home Price Index Monthly CHXRSA
ICE Swap Rates, 11:00 A.M. (London Time), Based on U.S. Dollar, 3 Year Tenor Daily ICERATES1100USD3Y
Coincident Economic Activity Index for the United States Monthly USPHCI
Housing Inventory: Median Listing Price per Square Feet in the United States Monthly MEDLISPRIPERSQUFEEUS
U.S. / New Zealand Foreign Exchange Rat

China / U.S. Foreign Exchange Rate Daily DEXCHUS
China / U.S. Foreign Exchange Rate Monthly EXCHUS
China / U.S. Foreign Exchange Rate Annual AEXCHUS
Gross Domestic Product for China Annual MKTGDPCNA646NWDB
Consumer Price Index: All Items for China Monthly CHNCPIALLMINMEI
Consumer Price Index: All Items for China Quarterly CHNCPIALLQINMEI
Consumer Price Index: All Items for China Annual CHNCPIALLAINMEI
M2 for China Monthly MYAGM2CNM189N
Current Price Gross Domestic Product in China Quarterly CHNGDPNQDSMEI
Real GDP at Constant National Prices for China Annual RGDPNACNA666NRUG
Interest Rates, Discount Rate for China Monthly INTDSRCNM193N
Current Price Gross Domestic Product in China Annual CHNGDPNADSMEI
Real Residential Property Prices for China Quarterly QCNR628BIS
Real Residential Property Prices for China Quarterly QCNR368BIS
U.S. Imports of Goods by Customs Basis from China Monthly IMPCH
Constant GDP per capita for China Annual NYGDPPCAPKDCHN
Total Reserves excluding Gold for China Mo

Output-side Real GDP at chained Purchasing Power Parities for China Annual RGDPOSCNA666NRUG
External Trade: Volume of Exports of Goods and Services for China, P.R.: Hong Kong Annual HKGTXRPCPPPT
Consumer opinion surveys: Economic Situation: Future tendency: National indicator for China Monthly CHNCSESFT02STSAM
Import Price Index by Origin (NAICS): Household and Institutional Furniture and Kitchen Cabinet Manufacturing for China Monthly COCHNZ3371
Share of Household Consumption at Current Purchasing Power Parities for Taiwan Province of China Annual CSHCCPTWA156NRUG
Total Credit to Private Non-Financial Sector, Unadjusted for Breaks, for China Quarterly, End of Quarter CRDQCNAPUBIS
Trade Weighted U.S. Dollar Index: Broad, Goods (DISCONTINUED) Monthly TWEXBMTH
Capital Accounts and Financial Accounts: Total Balance Including Change in Reserve Assets for China (DISCONTINUED) Annual BPCFTT01CNA637N
Reserve Assets for China (DISCONTINUED) Annual XFORSD01CNA194N
Reserve Assets for China (DISC

Secure Internet Servers for China Annual ITNETSECRP6CHN
Geographical Outreach: Key Indicators Commercial Bank Branches Per 100,000 Adults for China, P.R.: Macao Annual MACFCBODCANUM
Use of Financial Services Deposit Accounts: Accounts at Other Depository Corporations for China, P.R.: Mainland Annual CHNFCAODNUM
Amount Outstanding Due within One Year of International Debt Securities for Financial Corporations Sector, Residence of Issuer in China (DISCONTINUED) Quarterly IDS1YMAORIFCCN
Geographical Outreach: Number of Institutions, Other Financial Corporations, Other Financial Intermediaries for China, P.R.: Macao Annual MACFCIOFMNUM
Business Tendency Surveys for Services: Business Situation Activity: Present: European Commission Indicator for China (DISCONTINUED) Quarterly BVBUCT02CNQ460N
Value of Exports to China from West Virginia Annual WVCHNA052SCEN
Value of Exports to China from New Jersey Annual NJCHNA052SCEN
Use of Financial Services: Number of Depositors which are Households at 

In [14]:
working_age_population

{'Germany': {'Germany': {'title': 'Working Age Population: Aged 15-64: All Persons for Germany',
   'units': 'lin',
   'notes': 'OECD descriptor ID: LFWA64TT\nOECD unit ID: STSA\nOECD country ID: DEU\n\nAll OECD data should be cited as follows: OECD, "Main Economic Indicators - complete database", Main Economic Indicators (database),http://dx.doi.org/10.1787/data-00052-en (Accessed on date)\nCopyright, 2016, OECD. Reprinted with permission.',
   'dates': ['2005-01-01',
    '2006-01-01',
    '2007-01-01',
    '2008-01-01',
    '2009-01-01',
    '2010-01-01',
    '2011-01-01',
    '2012-01-01',
    '2013-01-01',
    '2014-01-01',
    '2015-01-01',
    '2016-01-01',
    '2017-01-01',
    '2018-01-01',
    '2019-01-01'],
   'values': ['54764150.0',
    '54543175.0',
    '54228900.0',
    '54065975.0',
    '53762500.0',
    '52399600.0',
    '52314000',
    '52486950.0',
    '52576625.0',
    '52729325.0',
    '52963825.0',
    '53802300.0',
    '53797375.0',
    '53523600.0',
    '53545050

In [15]:
calculated_working_age_population = []
calculated_working_age_dates = []
for date in working_age_population['China']['China']['dates']:
    if date in total_population_set['China']['China']['dates']:
        calculated_working_age_dates.append(date)
        working_population_index = working_age_population['China']['China']['dates'].index(date)
        total_population_index = total_population_set['China']['China']['dates'].index(date)
        print(date, working_population_index, total_population_index)
        total_population_value = total_population_set['China']['China']['values'][total_population_index]
        working_population_value = working_age_population['China']['China']['values'][working_population_index]
        print(total_population_value, working_population_value)
        if  working_population_value.replace('.','',1).isnumeric() and working_population_value.replace('.','',1).isnumeric():
            print(total_population_value,working_population_value )
            calculated_working_age_population.append(
                str( float(total_population_value) * (float(working_population_value) /100)  ))
        else:
            calculated_working_age_population.append('')
        
        
     

1952-01-01 2 0
579.5771775253429 .
1953-01-01 3 1
589.316980637563 .
1954-01-01 4 2
599.9429088684137 .
1955-01-01 5 3
612.0530597305674 .
1956-01-01 6 4
624.5681609057584 .
1957-01-01 7 5
638.7702985074699 .
1958-01-01 8 6
652.3771708474442 .
1959-01-01 9 7
660.0906215569922 .
1960-01-01 10 8
656.3693516817199 56.4892986405363
656.3693516817199 56.4892986405363
1961-01-01 11 9
650.3261901757719 56.4143062276544
650.3261901757719 56.4143062276544
1962-01-01 12 10
659.0341303875521 56.0432071621243
659.0341303875521 56.0432071621243
1963-01-01 13 11
680.1645079124097 55.5933222615383
680.1645079124097 55.5933222615383
1964-01-01 14 12
702.172144379058 55.3484291751092
702.172144379058 55.3484291751092
1965-01-01 15 13
721.8245794812078 55.3978086362411
721.8245794812078 55.3978086362411
1966-01-01 16 14
742.3605537337432 55.1412106991901
742.3605537337432 55.1412106991901
1967-01-01 17 15
761.9472313576623 55.220927981574
761.9472313576623 55.220927981574
1968-01-01 18 16
782.9626564369

In [16]:
calculated_working_age_population

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '370.7784432564387',
 '366.87700840439805',
 '369.34386296220015',
 '378.1260467923521',
 '388.64125201898844',
 '399.87499923035153',
 '409.34659708199774',
 '420.7543318856117',
 '434.4161987350207',
 '448.89868302299186',
 '462.17194200000034',
 '473.25952600000016',
 '484.4325410000003',
 '495.5718959999997',
 '507.08688799999965',
 '519.3825450000002',
 '532.7904419999994',
 '546.715142',
 '561.4578909999996',
 '577.2093619999997',
 '593.9544660000003',
 '612.3246949999999',
 '630.586798',
 '649.1697179999998',
 '668.3287140000002',
 '687.880535',
 '705.9792410000002',
 '724.1925330000007',
 '741.8451109999995',
 '758.5328460000002',
 '774.1555040000001',
 '784.3712019999997',
 '796.5042489999997',
 '809.1305570000003',
 '820.3267410000001',
 '829.4750910000004',
 '843.4635100000005',
 '852.6928590000006',
 '859.9841870000001',
 '869.4325729999997',
 '882.7293209999997',
 '894.395054',
 '910.4166209999996',
 '928.8457700000005',
 '946.12264

In [17]:
working_age_population['China']['China']['values'] = calculated_working_age_population
working_age_population['China']['China']['dates'] = calculated_working_age_dates

In [18]:
working_age_population['China']['China']['dates']

['1952-01-01',
 '1953-01-01',
 '1954-01-01',
 '1955-01-01',
 '1956-01-01',
 '1957-01-01',
 '1958-01-01',
 '1959-01-01',
 '1960-01-01',
 '1961-01-01',
 '1962-01-01',
 '1963-01-01',
 '1964-01-01',
 '1965-01-01',
 '1966-01-01',
 '1967-01-01',
 '1968-01-01',
 '1969-01-01',
 '1970-01-01',
 '1971-01-01',
 '1972-01-01',
 '1973-01-01',
 '1974-01-01',
 '1975-01-01',
 '1976-01-01',
 '1977-01-01',
 '1978-01-01',
 '1979-01-01',
 '1980-01-01',
 '1981-01-01',
 '1982-01-01',
 '1983-01-01',
 '1984-01-01',
 '1985-01-01',
 '1986-01-01',
 '1987-01-01',
 '1988-01-01',
 '1989-01-01',
 '1990-01-01',
 '1991-01-01',
 '1992-01-01',
 '1993-01-01',
 '1994-01-01',
 '1995-01-01',
 '1996-01-01',
 '1997-01-01',
 '1998-01-01',
 '1999-01-01',
 '2000-01-01',
 '2001-01-01',
 '2002-01-01',
 '2003-01-01',
 '2004-01-01',
 '2005-01-01',
 '2006-01-01',
 '2007-01-01',
 '2008-01-01',
 '2009-01-01',
 '2010-01-01',
 '2011-01-01',
 '2012-01-01',
 '2013-01-01',
 '2014-01-01',
 '2015-01-01',
 '2016-01-01',
 '2017-01-01',
 '2018-01-

In [19]:
#working_age_population_ratio

In [20]:
from collections import defaultdict

In [21]:
def make_dataframe(dict_info):
    columns = dict_info.keys()
    values = defaultdict(list)
    dates_set = set()
    for country in columns:
        #print(country)
        #print(dict_info[country])
        #print(dict_info[country][country])
        dates_set.update(set(dict_info[country][country]['dates']))
   
    print (dates_set)
    
    dates_list = list(dates_set)
    dates_list = sorted(dates_list)
    for date in dates_list:
        for country in dict_info:
            print(date, country)
            if date in dict_info[country][country]['dates']:
                values[date].append(
                    dict_info[country][country]['values'][dict_info[country][country]['dates'].index(date)])
            else:
                values[date].append('')
    print(values)
    df_ans = pd.DataFrame.from_dict(values, orient='index', columns=columns)         
    return df_ans
    

In [22]:
df_countries = make_dataframe(working_age_population)

{'1970-01-01', '2005-01-01', '1961-01-01', '1976-01-01', '1953-01-01', '2014-01-01', '2006-01-01', '2015-01-01', '1977-01-01', '2016-01-01', '1966-01-01', '2012-01-01', '2008-01-01', '1963-01-01', '1989-01-01', '1987-01-01', '2020-01-01', '1990-01-01', '2007-01-01', '1979-01-01', '1973-01-01', '1971-01-01', '1982-01-01', '1964-01-01', '2004-01-01', '1967-01-01', '2018-01-01', '1956-01-01', '1994-01-01', '1958-01-01', '1997-01-01', '1999-01-01', '2003-01-01', '1984-01-01', '1968-01-01', '1981-01-01', '1988-01-01', '2009-01-01', '1998-01-01', '1962-01-01', '1965-01-01', '1955-01-01', '1972-01-01', '1960-01-01', '2010-01-01', '1992-01-01', '2019-01-01', '1991-01-01', '2001-01-01', '1954-01-01', '2000-01-01', '1980-01-01', '2002-01-01', '2013-01-01', '1986-01-01', '1952-01-01', '1995-01-01', '1993-01-01', '1975-01-01', '1978-01-01', '2011-01-01', '1983-01-01', '1959-01-01', '1974-01-01', '1957-01-01', '2017-01-01', '1969-01-01', '1996-01-01', '1985-01-01'}
1952-01-01 Germany
1952-01-01 Sou

In [23]:

df_countries.rename(columns={'USA':'United States'}, inplace=True)

In [24]:
# Working age population
df_countries

,Germany,South Korea,Japan,United States,China
1952-01-01,,,,,
1953-01-01,,,,,
1954-01-01,,,,,
1955-01-01,,,,.,
1956-01-01,,,,.,
...,...,...,...,...,...
2016-01-01,53802300.0,36838591.6666667,76496666.6666666,205502750.0,1020.9825570000002
2017-01-01,53797375.0,36859450.0,75985833.3333333,205537500.0,1019.1215649999992
2018-01-01,53523600.0,36796125.0,75454166.6666667,206507750.0,1016.5153999999998
2019-01-01,53545050.0,36791233.3333333,75020833.3333334,206269916.666667,1014.0191179999998


In [25]:
df_countries.head()

,Germany,South Korea,Japan,United States,China
1952-01-01,,,,,
1953-01-01,,,,,
1954-01-01,,,,,
1955-01-01,,,,.,
1956-01-01,,,,.,


In [26]:
for column in list(df_countries.columns):
    df_countries[column] = pd.to_numeric(df_countries[column],  errors='coerce')
    print(column)
    if column != 'China':
        df_countries[column] = df_countries[column] / 1000000


Germany
South Korea
Japan
United States
China


In [27]:
df_countries_total = make_dataframe(total_population_set)

KeyError: 'Republic of Korea'

In [ ]:
df_countries_total.rename(columns={'Republic of Korea': 'South Korea'}, inplace=True)

In [ ]:
df_countries_total

In [ ]:
for column in list(df_countries_total.columns):
    df_countries_total[column] = pd.to_numeric(df_countries_total[column],  errors='coerce')
    print(column)
    if column == 'China':
        df_countries_total[column] = df_countries_total[column]
    else:
        df_countries_total[column] = df_countries_total[column] / 1000

In [ ]:
# Change year labels to only year
df_countries['Year'] = df_countries.index.map(lambda x: x.split('-')[0])
df_countries_total['Year'] = df_countries_total.index.map(lambda x: x.split('-')[0])

In [ ]:
df_countries.set_index(['Year'], inplace=True)
df_countries_total.set_index(['Year'], inplace=True)

In [ ]:
data_dir = '../data'

In [ ]:
import os
pop_file = 'FRED_pop.csv'
work_file = 'FRED_work.csv'
pop_path = os.path.join(data_dir, pop_file)
work_path = os.path.join(data_dir, work_file)

In [ ]:
df_countries.to_csv(work_path, index=True)
df_countries_total.to_csv(pop_path, index=True)

In [ ]:
#

In [ ]:
df_countries

In [ ]:
df_countries.plot(title='Working Age Population')

In [ ]:
df_temp = df_countries.copy()

In [ ]:
# Normalize data
df_temp2 = (df_temp - df_temp.min()) / (df_temp.max() - df_temp.min())

In [ ]:
df_temp2.plot(title='Working Age Population  - normalized')

In [ ]:
df_temp3 = (df_temp / df_temp.max()) * 100

In [ ]:
df_temp3.plot(title='Working Age Population  - max = 100%')

In [ ]:
df_countries_total.plot(title='Total Population')

In [ ]:
df_countries_total